In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [19]:
df2016 = pd.read_csv('data/rtc_2016.csv')
df2017 = pd.read_csv('data/rtc_2017.csv')
df2018 = pd.read_csv('data/rtc_2018.csv')
df2019 = pd.read_csv('data/rtc_2019.csv')
df2020 = pd.read_csv('data/rtc_2020.csv')

#### Aggregate Data

In [46]:
total_df = pd.concat([df2016,df2017,df2018,df2019,df2020], ignore_index=True)

In [47]:
total_df.drop('Unnamed: 0', inplace=True, axis=1)

##### Time Splitter Function

In [48]:
new_crash_time = total_df['crash_time'].astype(str).apply(lambda x: re.sub("[^0-9]", "", x))

In [49]:
new_arrival_time = total_df['arrival_time'].astype(str).apply(lambda x: re.sub("[^0-9]", "", x))

In [50]:
new_response_time = total_df['response_time'].astype(str).apply(lambda x: re.sub("[^0-9]", "", x))

In [51]:
total_df['crash_time'] = pd.to_numeric(new_crash_time)

In [52]:
total_df['arrival_time'] = pd.to_numeric(new_arrival_time)

In [53]:
total_df['response_time'] = pd.to_numeric(new_response_time)

In [54]:
total_df.to_csv('data/aggregated.csv')

#### EDA

In [57]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 911 entries, 0 to 910
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   sn                        911 non-null    int64 
 1   command                   911 non-null    object
 2   date                      911 non-null    object
 3   crash_time                911 non-null    int64 
 4   report_time               911 non-null    int64 
 5   arrival_time              911 non-null    int64 
 6   response_time             911 non-null    int64 
 7   route                     911 non-null    object
 8   location                  911 non-null    object
 9   vehicle_no                911 non-null    object
 10  vehicle_type              910 non-null    object
 11  vehicle_cat               911 non-null    object
 12  vehicle_make              910 non-null    object
 13  vehicle_model             910 non-null    object
 14  fleet_operator            